In [1]:
import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
from collections import Counter
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# LATIMES 1914 -2002

In [2]:
years = range(1914, 2022)
months = range(1,13)
products = itertools.product(years,months)
df= pd.DataFrame([ [year, month] for (year,month) in products],columns=['year', 'month'])
df['page'] = 'https://www.latimes.com/sitemap/' + df['year'].astype(str) + '/' + df['month'].astype(str)
df

,year,month,page
0,1914,1,https://www.latimes.com/sitemap/1914/1
1,1914,2,https://www.latimes.com/sitemap/1914/2
2,1914,3,https://www.latimes.com/sitemap/1914/3
3,1914,4,https://www.latimes.com/sitemap/1914/4
4,1914,5,https://www.latimes.com/sitemap/1914/5
...,...,...,...
1291,2021,8,https://www.latimes.com/sitemap/2021/8
1292,2021,9,https://www.latimes.com/sitemap/2021/9
1293,2021,10,https://www.latimes.com/sitemap/2021/10
1294,2021,11,https://www.latimes.com/sitemap/2021/11


In [56]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

def get_other_page_links(page):
    try:
        coverpage = requests.get(page,headers = headers).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        current_body = soup.find_all('ul', {'class':'archive-page-menu'})[0]
#         current_href = current_body.find_all('a', href= True)
        more_pages = soup.find_all('ul', {'class':'archive-page-pagination-menu'})

        if len(more_pages):
            more_href = more_pages[0].find_all('a', href=True)
            more_href = [page + i['href'] for i in more_href]
        else:
            more_href = []
    
        return more_href
    except:
        return 'None'

In [57]:
pandarallel.initialize(nb_workers=8, progress_bar=True)
df['more_pages']=None
df['more_pages']= df['page'].parallel_apply(get_other_page_links).values

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [65]:
df = df[df.more_pages!='None'].reset_index(drop=True)
df

,year,month,page,more_pages
0,1914,12,https://www.latimes.com/sitemap/1914/12,[https://www.latimes.com/sitemap/1914/12#]
1,1915,5,https://www.latimes.com/sitemap/1915/5,[https://www.latimes.com/sitemap/1915/5#]
2,1919,10,https://www.latimes.com/sitemap/1919/10,[https://www.latimes.com/sitemap/1919/10#]
3,1923,3,https://www.latimes.com/sitemap/1923/3,[https://www.latimes.com/sitemap/1923/3#]
4,1926,11,https://www.latimes.com/sitemap/1926/11,[https://www.latimes.com/sitemap/1926/11#]
...,...,...,...,...
559,2020,12,https://www.latimes.com/sitemap/2020/12,"[https://www.latimes.com/sitemap/2020/12#, https://www.latimes.com/sitemap/2020/12?p=2, https://www.latimes.com/sitemap/2020/12?p=3, https://www.latimes.com/sitemap/2020/12?p=4, https://www.latimes.com/sitemap/2020/12?p=5, https://www.latimes.com/sitemap/2020/12?p=6, https://www.latimes.com/sitemap/2020/12?p=7, https://www.latimes.com/sitemap/2020/12?p=8, https://www.latimes.com/sitemap/2020/12?p=9, https://www.latimes.com/sitemap/2020/12?p=10, https://www.latimes.com/sitemap/2020/12?p=11, https://www.latimes.com/sitemap/2020/12?p=12]"
560,2021,1,https://www.latimes.com/sitemap/2021/1,"[https://www.latimes.com/sitemap/2021/1#, https://www.latimes.com/sitemap/2021/1?p=2, https://www.latimes.com/sitemap/2021/1?p=3, https://www.latimes.com/sitemap/2021/1?p=4, https://www.latimes.com/sitemap/2021/1?p=5, https://www.latimes.com/sitemap/2021/1?p=6, https://www.latimes.com/sitemap/2021/1?p=7, https://www.latimes.com/sitemap/2021/1?p=8, https://www.latimes.com/sitemap/2021/1?p=9, https://www.latimes.com/sitemap/2021/1?p=10, https://www.latimes.com/sitemap/2021/1?p=11, https://www.latimes.com/sitemap/2021/1?p=12]"
561,2021,2,https://www.latimes.com/sitemap/2021/2,"[https://www.latimes.com/sitemap/2021/2#, https://www.latimes.com/sitemap/2021/2?p=2, https://www.latimes.com/sitemap/2021/2?p=3, https://www.latimes.com/sitemap/2021/2?p=4, https://www.latimes.com/sitemap/2021/2?p=5, https://www.latimes.com/sitemap/2021/2?p=6, https://www.latimes.com/sitemap/2021/2?p=7, https://www.latimes.com/sitemap/2021/2?p=8, https://www.latimes.com/sitemap/2021/2?p=9, https://www.latimes.com/sitemap/2021/2?p=10, https://www.latimes.com/sitemap/2021/2?p=11]"
562,2021,3,https://www.latimes.com/sitemap/2021/3,"[https://www.latimes.com/sitemap/2021/3#, https://www.latimes.com/sitemap/2021/3?p=2, https://www.latimes.com/sitemap/2021/3?p=3, https://www.latimes.com/sitemap/2021/3?p=4, https://www.latimes.com/sitemap/2021/3?p=5, https://www.latimes.com/sitemap/2021/3?p=6, https://www.latimes.com/sitemap/2021/3?p=7, https://www.latimes.com/sitemap/2021/3?p=8, https://www.latimes.com/sitemap/2021/3?p=9, https://www.latimes.com/sitemap/2021/3?p=10, https://www.latimes.com/sitemap/2021/3?p=11, https://www.latimes.com/sitemap/2021/3?p=12, https://www.latimes.com/sitemap/2021/3?p=13]"


In [66]:
df = df.explode('more_pages').reset_index(drop=True)
df

,year,month,page,more_pages
0,1914,12,https://www.latimes.com/sitemap/1914/12,https://www.latimes.com/sitemap/1914/12#
1,1915,5,https://www.latimes.com/sitemap/1915/5,https://www.latimes.com/sitemap/1915/5#
2,1919,10,https://www.latimes.com/sitemap/1919/10,https://www.latimes.com/sitemap/1919/10#
3,1923,3,https://www.latimes.com/sitemap/1923/3,https://www.latimes.com/sitemap/1923/3#
4,1926,11,https://www.latimes.com/sitemap/1926/11,https://www.latimes.com/sitemap/1926/11#
...,...,...,...,...
12084,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=4
12085,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=5
12086,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=6
12087,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=7


In [67]:
df.to_csv('latimes_1.csv', index=False)

In [78]:
get_current_page_links('https://www.latimes.com/sitemap/2021/4?p=8')

['https://www.latimes.com/entertainment-arts/music/pf-si60k9nm-123',
 'https://www.latimes.com/entertainment-arts/music/gs069dndiyk-123',
 'https://www.latimes.com/entertainment-arts/music/bw8-brnbv-q-123',
 'https://www.latimes.com/california/story/2021-04-01/mass-vaccination-site-at-cal-state-l-a-will-stay-open-managed-by-the-city-of-l-a',
 'https://www.latimes.com/sports/ucla/story/2021-04-01/ucla-basketball-march-madness-martin-jarmond',
 'https://www.latimes.com/sports/chargers/story/2021-04-01/jeff-bezos-chargers',
 'https://www.latimes.com/california/story/2021-04-01/california-new-attorney-general-faces-major-challenges-rob-bonta',
 'https://www.latimes.com/business/story/2021-04-01/wall-street-closes-higher-pushing-s-p-500-past-4-000-points',
 'https://www.latimes.com/california/story/2021-04-01/half-of-california-bachelors-degrees-are-awarded-by-cal-state-report-says',
 'https://www.latimes.com/sports/dodgers/story/2021-04-01/dodgers-cody-bellinger-is-robbed-of-home-run-by-te

In [72]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

def get_current_page_links(page):
    try:
        coverpage = requests.get(page,headers = headers).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        body = soup.find_all('ul', {'class':'archive-page-menu'})[0]
        href = body.find_all('a', href= True)
        return [i['href'] for i in href]
    except:
        print(page)
        return

In [84]:
pandarallel.initialize(nb_workers=32, progress_bar=False)
batch_size = 32*5
i=0
while i < df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1,'links']=\
    df.loc[i: i+ batch_size-1, 'more_pages'].parallel_apply(get_current_page_links).values
    i += batch_size

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,4320,4480,4640,4800,4960,5120,5280,5440,5600,5760,5920,6080,6240,6400,6560,6720,6880,7040,7200,7360,7520,7680,7840,8000,8160,8320,8480,8640,8800,8960,9120,9280,9440,9600,9760,9920,10080,10240,10400,10560,10720,10880,11040,11200,https://www.latimes.com/sitemap/2017/3?p=2
11360,11520,11680,11840,12000,

In [98]:
df.to_csv('latimes_2.csv', index=False)

In [2]:
df= pd.read_csv('latimes_2.csv', header=0)
df

,year,month,page,more_pages,links
0,1914,12,https://www.latimes.com/sitemap/1914/12,https://www.latimes.com/sitemap/1914/12#,['https://www.latimes.com/local/obituaries/archives/la-me-john-muir-19141225-story.html']
1,1915,5,https://www.latimes.com/sitemap/1915/5,https://www.latimes.com/sitemap/1915/5#,['https://www.latimes.com/local/obituaries/archives/la-me-ac-bilicke-19150508-story.html']
2,1919,10,https://www.latimes.com/sitemap/1919/10,https://www.latimes.com/sitemap/1919/10#,['https://www.latimes.com/entertainment/arts/laphil/la-et-cm-la-phil-review-walter-rothwell-19191025-story.html']
3,1923,3,https://www.latimes.com/sitemap/1923/3,https://www.latimes.com/sitemap/1923/3#,['https://www.latimes.com/local/obituaries/archives/la-me-sarah-bernhardt-19230328-story.html']
4,1926,11,https://www.latimes.com/sitemap/1926/11,https://www.latimes.com/sitemap/1926/11#,"['https://www.latimes.com/local/obituaries/archives/la-me-harry-houdini-19261101-story.html', 'https://www.latimes.com/la-me-harry-houdinis-death-20151026-htmlstory.html']"
...,...,...,...,...,...
12084,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=4,"['https://www.latimes.com/entertainment-arts/story/2021-04-12/prince-philip-death-meghan-markle-prince-harry-statement', 'https://www.latimes.com/sports/highschool/story/2021-04-12/cif-southern-section-will-hold-playoffs-spring-sports', 'https://www.latimes.com/california/story/2021-04-12/covid-vaccine-appointments-expand-chinatown-east-la', 'https://www.latimes.com/california/story/2021-04-12/what-to-know-la-opens-covid-19-vaccine-16-and-up', 'https://www.latimes.com/entertainment-arts/awards/story/2021-04-12/two-strangers-come-together-with-the-personal-and-particular-with-minari', 'http..."
12085,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=5,"['https://www.latimes.com/business/story/2021-04-09/amazon-union-vote', 'https://www.latimes.com/lifestyle/story/2021-04-09/hiking-after-miscarriage-how-la-trails-eased-grief', 'https://www.latimes.com/travel/story/2021-04-09/palm-springs-driving-tour', 'https://www.latimes.com/california/story/2021-04-09/burbank-restaurant-repeatedly-defies-covid-closure-orders', 'https://www.latimes.com/sports/dodgers/liveblog/dodgers-washington-nationals-live-updates-news-score-mlb', 'https://www.latimes.com/sports/story/2021-04-09/everybody-wants-to-have-a-hero-fernandomania-at-40', 'https://www.latime..."
12086,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=6,"['https://www.latimes.com/opinion/story/2021-04-07/china-xinjiang-human-rights-sanctions-boycotts', 'https://www.latimes.com/opinion/story/2021-04-07/census-differential-privacy-trump-biden-congress', 'https://www.latimes.com/opinion/story/2021-04-07/mountain-lions-killed-on-roads', 'https://www.latimes.com/opinion/story/2021-04-07/boyle-heights-sears-ticket-to-the-middle-class', 'https://www.latimes.com/opinion/story/2021-04-07/gop-vaccine-passports-womens-rights-abortion', 'https://www.latimes.com/opinion/story/2021-04-07/pandemic-close-educational-divides', 'https://www.latimes.com/opin..."
12087,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=7,"['https://www.latimes.com/world-nation/story/2021-04-04/jordan-former-crown-prince-accused-of-targeting-kingdom-stability', 'https://www.latimes.com/sports/ucla/story/2021-04-04/buzzer-beaters-during-ucla-coach-mick-cronins-tenure-win-some-lose-some', 'https://www.latimes.com/sports/highschool/story/2021-04-04/la-times-high-school-baseball-rankings', 'https://www.latimes.com/entertainment-arts/tv/story/2021-04-04/snl-daniel-kaluuya-matt-gaetz-lil-nas-x-david-dobrik', 'https://www.latimes.com/entertainment-arts/uvo-6rgsiyi-123', 'https://www.latimes.com/entertainment-arts/vu1awct1cps-123', ..."


# step 2

In [3]:
df.links = df.links.apply(eval)
df = df.explode('links').reset_index(drop=True)
df

,year,month,page,more_pages,links
0,1914,12,https://www.latimes.com/sitemap/1914/12,https://www.latimes.com/sitemap/1914/12#,https://www.latimes.com/local/obituaries/archives/la-me-john-muir-19141225-story.html
1,1915,5,https://www.latimes.com/sitemap/1915/5,https://www.latimes.com/sitemap/1915/5#,https://www.latimes.com/local/obituaries/archives/la-me-ac-bilicke-19150508-story.html
2,1919,10,https://www.latimes.com/sitemap/1919/10,https://www.latimes.com/sitemap/1919/10#,https://www.latimes.com/entertainment/arts/laphil/la-et-cm-la-phil-review-walter-rothwell-19191025-story.html
3,1923,3,https://www.latimes.com/sitemap/1923/3,https://www.latimes.com/sitemap/1923/3#,https://www.latimes.com/local/obituaries/archives/la-me-sarah-bernhardt-19230328-story.html
4,1926,11,https://www.latimes.com/sitemap/1926/11,https://www.latimes.com/sitemap/1926/11#,https://www.latimes.com/local/obituaries/archives/la-me-harry-houdini-19261101-story.html
...,...,...,...,...,...
3521782,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/opinion/story/2021-04-01/editorial-pandemic-isnt-over-here-until-its-over-everywhere
3521783,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/opinion/story/2021-04-01/autopsy-records-bill-press-access
3521784,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/california/story/2021-04-01/sheriff-villanueva-will-not-reveal-cause-tiger-woods-crash
3521785,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/world-nation/story/2021-04-01/hong-kong-democracy-leaders-convicted-2019-protests


In [113]:
df[df.links.isnull()]

,year,month,page,more_pages,links


In [2]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [5]:
df['title_in'], df['title_out'] = None, None

In [6]:


df['title_in'] = df['links'].parallel_apply(find_relevant)
df['title_out'] = df['links'].parallel_apply(find_exclude)

head_in = df['title_in'].str.len()
head_out = df['title_out'].str.len()

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df[remove_index== True].index
remove_index.shape

(505224,)

In [7]:
df = df[~df.index.isin(remove_index)].reset_index(drop=True)
df.to_csv('latimes_3.csv', index=False)

In [21]:
df= pd.read_csv('latimes_3.csv', header=0)
df

,year,month,page,more_pages,links,title_in,title_out
0,1926,11,https://www.latimes.com/sitemap/1926/11,https://www.latimes.com/sitemap/1926/11#,https://www.latimes.com/la-me-harry-houdinis-death-20151026-htmlstory.html,{'death': 1},{}
1,1927,9,https://www.latimes.com/sitemap/1927/9,https://www.latimes.com/sitemap/1927/9#,https://www.latimes.com/la-isadora-duncans-death-20141120-htmlstory.html,{'death': 1},{}
2,1930,8,https://www.latimes.com/sitemap/1930/8,https://www.latimes.com/sitemap/1930/8#,https://www.latimes.com/la-lon-chaneys-death-20150817-htmlstory.html,{'death': 1},{}
3,1935,7,https://www.latimes.com/sitemap/1935/7,https://www.latimes.com/sitemap/1935/7#,https://www.latimes.com/la-william-mulhollands-death-htmlstory.html,{'death': 1},{}
4,1936,1,https://www.latimes.com/sitemap/1936/1,https://www.latimes.com/sitemap/1936/1#,https://www.latimes.com/la-me-john-gilberts-death-20160104-htmlstory.html,{'death': 1},{}
...,...,...,...,...,...,...,...
3016558,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/opinion/story/2021-04-01/editorial-pandemic-isnt-over-here-until-its-over-everywhere,{},{}
3016559,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/opinion/story/2021-04-01/autopsy-records-bill-press-access,{},{}
3016560,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/california/story/2021-04-01/sheriff-villanueva-will-not-reveal-cause-tiger-woods-crash,{},{}
3016561,2021,4,https://www.latimes.com/sitemap/2021/4,https://www.latimes.com/sitemap/2021/4?p=8,https://www.latimes.com/world-nation/story/2021-04-01/hong-kong-democracy-leaders-convicted-2019-protests,"{'convict': 1, 'protest': 1}",{}


In [22]:
num_chunks = 30
df_save = np.array_split(df[['year', 'month','links']], num_chunks)
save_chunks = lambda x: df_save[x].to_csv('latimes/%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
del df_save

In [2]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        article.download()
        article.parse()
        return article.text
    except:
        print('error %s'%url)
        return

In [ ]:
pandarallel.initialize(nb_workers=40, progress_bar=True)
for idx in range(100):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('latimes/%03d.csv'%idx, header=0)
    df['text']=None
    df['text']= df['links'].parallel_apply(extract_story).values
    df.to_csv('latimes/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------0 ------------


error https://www.latimes.com/archives/la-xpm-1985-03-29-me-20319-story.html
error https://www.latimes.com/archives/la-xpm-1985-04-19-mn-14940-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-10-06-re-5461-story.html

error https://www.latimes.com/archives/la-xpm-1985-08-08-fi-3534-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-02-28-mn-12740-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-03-07-fi-34649-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-07-04-ca-9305-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-09-15-mn-23091-story.html




error https://www.latimes.com/archives/la-xpm-1985-02-10-sp-3509-story.html
error https://www.latimes.com/archives/la-xpm-1985-09-15-mn-23143-story.htmlerror https://www.latimes.com/archives/la-xpm-1985-09-22-bk-17912-story.html

error https://www.latimes.com/archives/la-xpm-1985-09-30-vw-17707-story.html

 ----------1 ------------


error https://www.latimes.com/archives/la-xpm-1985-12-16-me-782-story.html
error https://www.latimes.com/archives/la-xpm-1986-04-10-ga-3468-story.html

 ----------2 ------------


error https://www.latimes.com/archives/la-xpm-1987-02-15-me-3518-story.html

 ----------3 ------------


error https://www.latimes.com/archives/la-xpm-1988-02-15-mn-28973-story.html

 ----------4 ------------


...

error https://www.latimes.com/archives/la-xpm-2006-feb-21-le-tuesday21.1-story.htmlerror https://www.latimes.com/archives/la-xpm-2006-apr-26-sp-nhlrdp26-story.htmlerror https://www.latimes.com/archives/la-xpm-2006-mar-10-sp-newswire10-story.html

error https://www.latimes.com/archives/la-xpm-2005-may-12-wk-letters12.2-story.html

error https://www.latimes.com/archives/la-xpm-2006-jan-04-fi-briefs4.2-story.html
error https://www.latimes.com/archives/la-xpm-2006-mar-22-le-wednesday22.4-story.html
error https://www.latimes.com/archives/la-xpm-2004-oct-03-fi-letters3.3-story.html
error https://www.latimes.com/world/cs-051015soxgamer-story.html
error https://www.latimes.com/archives/la-xpm-2006-apr-07-sp-hssondcol7-story.htmlerror https://www.latimes.com/archives/la-xpm-2005-sep-28-fo-review28-story.html

error https://www.latimes.com/archives/la-xpm-2005-may-21-fg-taiwan21-story.htmlerror https://www.latimes.com/archives/la-xpm-2005-sep-27-sp-insideusc27-story.html

error https://www.latim

error https://www.latimes.com/archives/la-xpm-2006-nov-22-me-parking22-story.htmlerror https://www.latimes.com/archives/la-xpm-2007-jan-26-fi-chiecon26-story.html

error https://www.latimes.com/archives/la-xpm-2006-dec-10-fg-briefs10.6-story.htmlerror https://www.latimes.com/archives/la-xpm-2007-sep-03-le-monday3.s3-story.html

error https://www.latimes.com/archives/la-xpm-2006-nov-22-me-lopez22-story.html
error https://www.latimes.com/archives/la-xpm-2007-jul-15-me-paffenbarger15-story.htmlerror https://www.latimes.com/archives/la-xpm-2006-sep-20-sp-rydernotes20-story.htmlerror https://www.latimes.com/archives/la-xpm-2006-may-05-le-friday5.5-story.htmlerror https://www.latimes.com/archives/la-xpm-2007-aug-30-sp-briefing30-story.html



error https://www.latimes.com/archives/la-xpm-2006-may-19-me-only19-story.html

 ----------25 ------------


error https://www.latimes.com/opinion/la-oe-kerry30-2009mar30-story.html

 ----------26 ------------


error https://www.latimes.com/archives/la-xpm-2010-feb-01-la-fi-calpers-glancebox1-2010feb01-story.htmlerror https://www.latimes.com/local/la-xpm-2011-jul-01-la-me-mongols-logo-20110701-story.html


 ----------27 ------------


error https://www.latimes.com/local/lanow/la-xpm-2013-nov-26-la-me-ln-us-border-patrol-agents-attacked-mexico-crowd-20131126-story.html
error https://www.latimes.com/science/sciencenow/la-xpm-2013-dec-12-la-sci-newtown-anniversary-psychologists-guns-20131212-story.htmlerror https://www.latimes.com/science/sciencenow/la-xpm-2014-apr-08-la-sci-sn-spinal-cord-stimulation-paralysis-20140407-story.htmlerror https://www.latimes.com/nation/nationnow/la-na-nn-ebola-doctor-condition-improving-0140815-story.htmlerror https://www.latimes.com/nation/la-na-government-shutdown-20130923-story.html



error https://www.latimes.com/local/lanow/la-me-ln-man-walking-covered-blood-arrested-20131024-story.htmlerror https://www.latimes.com/local/lanow/la-me-ln-man-held-molotov-cocktail-attack-church-20140129-story.html

error https://www.latimes.com/nation/la-na-0408-fort-hood-20140408-story.html
error https://www.latimes.com/world/la-xpm-2013-may-23-la-fg-afghanistan-woman-ceo-20130524-story.htmlerror http

In [3]:
pandarallel.initialize(nb_workers=40, progress_bar=True)
for idx in range(27,100):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('latimes/%03d.csv'%idx, header=0)
    df['text']=None
    df['text']= df['links'].parallel_apply(extract_story).values
    df.to_csv('latimes/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------27 ------------


error https://www.latimes.com/80902214-132.html

 ----------28 ------------


error https://www.latimes.com/local/california/la-me-0607-lopez-fallingapart-20150606-column.html
error https://www.latimes.com/82962015-132.html
error https://www.latimes.com/opinion/op-ed/la-oe-newton-column-lapd-drones-20141117-column.html
error https://www.latimes.com/87475342-132.html
error https://www.latimes.com/business/hiltzik/la-fi-mh-supreme-court-saves-obamacare-20150625-column.html
error https://www.latimes.com/local/california/la-me-cap-20150928-column.html
error https://www.latimes.com/local/politics/la-me-cap-confederate-flag-20150625-column.html
error https://www.latimes.com/opinion/op-ed/la-oe-0624-morrison-labonge-20150624-column.html
error https://www.latimes.com/85183090-132.html
error https://www.latimes.com/85183085-132.html
error https://www.latimes.com/local/politics/la-me-cap-roads-healthcare-20150622-column.html
error https://www.latimes.com/84756241-132.html
error https://www.latimes.com/la-test-live-blog-training-updates-test-headline-for-live-1485983647-ht

error https://www.latimes.com/116663390-132.html
error https://www.latimes.com/nation/la-na-jayme-closs-kidnapper-jake-patterson-sentenced-to-life-in-prison-20190524-story.html
error https://www.latimes.com/world/la-fg-north-korea-nuclear-talks-trump-kim-20190524-story.html
error https://www.latimes.com/opinion/op-ed/la-oe-graedon-abortion-legal-alabama-pregnant-20190523-story.html
error https://www.latimes.com/opinion/op-ed/la-oe-mohammed-stateless-manus-island-20180723-story.htmlerror https://www.latimes.com/nation/ct-kansas-waterslide-death-settlement-20170504-story.htmlerror https://www.latimes.com/nation/ct-gop-governors-news-site-20170919-story.html


error https://www.latimes.com/nation/ct-london-subway-bomb-suspects-20170918-story.html
error https://www.latimes.com/world-nation/story/2020-01-29/trump-shores-up-support-for-newest-gop-congressman-in-jersey
error https://www.latimes.com/california/story/2020-05-28/coronavirus-order-lifted-in-el-dorado-county-but-not-lake-tahoe
err

FileNotFoundError: [Errno 2] No such file or directory: 'latimes/030.csv'

In [9]:
pandarallel.initialize(nb_workers=8, progress_bar=True)
for idx in range(11,30):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('latimes/%03d.csv'%idx, header=0)
    if df[df.text.isnull()].shape[0]:
        df.loc[df.text.isnull(), 'text']= df.loc[df.text.isnull(), 'links'].parallel_apply(extract_story).values
        df.to_csv('latimes/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------11 ------------



 ----------12 ------------



 ----------13 ------------



 ----------14 ------------



 ----------15 ------------


...


 ----------26 ------------



 ----------27 ------------


error https://www.latimes.com/80902214-132.html
error https://www.latimes.com/visuals/photography/la-fg-israel-gaza-protests-pictures-photogallery.html
error https://www.latimes.com/local/lanow/la-the-great-american-shakeout-pictures-photogallery.htmlerror https://www.latimes.com/nation/nationnow/la-na-nn-shark-week-returns-20120813-photos-photogallery.html

error https://www.latimes.com/world/la-fg-ice-festival-pg-photogallery.html

 ----------28 ------------


error https://www.latimes.com/nation/ct-photos-violence-at-trump-rally-in-new-mexico-20160524-photogallery.html
error https://www.latimes.com/87189933-132.html
error https://www.latimes.com/87166797-132.html
error https://www.latimes.com/83646088-132.html
error https://www.latimes.com/la-trb-las-vegas-chocolate-20160126-photogallery.html
error https://www.latimes.com/83627015-132.html
error https://www.latimes.com/83626915-132.html
error https://www.latimes.com/84125052-132.html
error https://www.latimes.com/la-hm-motti-casa-pictures-photogallery.htmlerror https://www.latimes.com/politics/la-pol-ca-campaign-mailers-independent-expenditures-photos-photogallery.html

error https://www.latimes.com/local/california/la-me-cap-20150928-column.html
error https://www.latimes.com/87475342-132.html
error https://www.latimes.com/84525220-132.html
error https://www.latimes.com/84461253-132.html
error https://www.latimes.com/83483702-132.html
error https://www.latimes.com/85861023-132.html
error ht

error https://www.latimes.com/espanol/deportes/la-world-cup-espanol-updates-2018-htmlstory.html
error https://www.latimes.com/116663390-132.html
error https://www.latimes.com/nation/la-na-jayme-closs-kidnapper-jake-patterson-sentenced-to-life-in-prison-20190524-story.html
error https://www.latimes.com/world/la-fg-north-korea-nuclear-talks-trump-kim-20190524-story.html
error https://www.latimes.com/opinion/op-ed/la-oe-graedon-abortion-legal-alabama-pregnant-20190523-story.html
error https://www.latimes.com/local/lanow/la-pomona-officer-mourned-20180311-photos-photogallery.html
error https://www.latimes.com/112832160-132.html
error https://www.latimes.com/109532111-132.html
error https://www.latimes.com/opinion/enterthefray/la-ed-infrastructure-trump-pelosi-20190501-story.html
error https://www.latimes.com/la-me-edu-lausd-school-board-election-results-20190312-story.html
error https://www.latimes.com/local/lanow/la-me-ln-alonzo-leron-smith-sentencing-20190307-story.html


In [11]:
df.links.apply(lambda x: x.replace('https://www.latimes.com/', '').split('/')[0]).unique()

array(['sports', '92705433-132.html', 'local', ..., 'mskkfbedsxa-123',
       '00000178-af63-d23f-a9fe-bf7b68060000-123',
       'dodgerstadiumrenovations'], dtype=object)

In [25]:
pandarallel.initialize(nb_workers=40, progress_bar=False)
for idx in range(1,30):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('latimes/%03d.csv'%idx, header=0)
    df = df[df.text.str.len()>200]
    df = df[~df.text.duplicated()]
    df['date'] = df['year'].astype(str) + '-' + df['month'].astype(str)
    df = df.rename(columns={'links':'url'})
    df = df[['date','text','url']]
    df = df.sort_values('date')
    print(df.date[0],df.date[len(df)-1])
    df.to_csv('latimes/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------1 ------------
0    1985-12
Name: date, dtype: object 92801    1986-9
Name: date, dtype: object

 ----------2 ------------
0    1986-10
Name: date, dtype: object 100552    1987-7
Name: date, dtype: object

 ----------3 ------------
36358    1987-10
Name: date, dtype: object 100551    1988-4
Name: date, dtype: object

 ----------4 ------------
77522    1988-10
Name: date, dtype: object 58462    1988-9
Name: date, dtype: object

 ----------5 ------------
0    1988-12
Name: date, dtype: object 100551    1989-9
Name: date, dtype: object

 ----------6 ------------
19412    1989-10
Name: date, dtype: object 100551    1990-5
Name: date, dtype: object

 ----------7 ------------
72052    1990-10
Name: date, dtype: object 58162    1990-9
Name: date, dtype: object

 ----------8 ------------
0    1990-12
Name: date, dtype: object 100551    1991-

# filtering

In [8]:
for idx in range(30):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('latimes/%03d.csv'%idx, header=0)
    print(df.shape[0])
    head = df['url']+' '+ df['text'].parallel_apply(lambda x: x[0:200])
    
    head_in = head.parallel_apply(find_relevant).str.len()
    head_out = head.parallel_apply(find_exclude).str.len()

    text_in = df['text'].parallel_apply(find_relevant).str.len()
    text_out = df['text'].parallel_apply(find_relevant).str.len()

    remove_index =\
        ((head_in ==0) & ( head_out > 0)) |\
        ( head_out >1)|\
        ((text_in  >2) & ( text_out > text_in)) |\
        ( text_out <2)

    remove_index = df[remove_index== True].index
    remove_index.shape
    df = df[~df.index.isin(remove_index)].reset_index(drop=True)
    print(df.shape[0])
    print(df.date[0],df.date[len(df)-1])
    df.to_csv('latimes-1/%03d.csv'%idx, index=False)


 ----------0 ------------
74830
43701
1966-8 1985-7

 ----------1 ------------
94397
54038
1985-12 1986-9

 ----------2 ------------
94641
52812
1986-10 1987-7

 ----------3 ------------
95598
52730
1987-10 1988-4

 ----------4 ------------
95249
53841
1988-10 1988-9

 ----------5 ------------
95727
53546
1988-12 1989-9

 ----------6 ------------
95786
52197
1989-10 1990-5

 ----------7 ------------
95642
52290
1990-10 1990-9

 ----------8 ------------
94999
52799
1990-12 1991-8

 ----------9 ------------
95619
51384
1991-10 1992-5

 ----------10 ------------
95641
51570
1992-10 1993-2

 ----------11 ------------
97064
53182
1993-10 1993-9

 ----------12 ------------
97714
53685
1993-11 1994-9

 ----------13 ------------
98030
53593
1994-10 1995-8

 ----------14 ------------
97743
51600
1995-10 1996-7

 ----------15 ------------
97522
50074
1996-10 1997-6

 ----------16 ------------
97573
48247
1997-10 1998-5

 ----------17 ------------
96342
47130
1998-10 1999-4

 ----------18 ------

In [9]:
quit()